In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # Seaborn for data visualization

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Load the data
df = pd.read_csv('../input/spotify-music-data-to-identify-the-moods/data_moods.csv')
df

In [ ]:
df.columns

In [ ]:
# Checking null value in dataset
df.isnull().sum()

In [ ]:
# Visualize number of data point for each class
sns.countplot(df['mood'])

# Data Preparation

In [ ]:
# Feature engineering
X = df.loc[:, 'popularity':'time_signature']
X['length'] = X['length']/max(X['length'])

# Mapping class label to respected integer
y = df['mood'].map({'Happy': 0, 'Sad': 1, 'Energetic': 2, 'Calm':3})
target_names = ['Happy', 'Sad', 'Energetic', 'Calm']

In [ ]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# Splitting training data and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
models = []
models.append(('Random Forest Classifier',RandomForestClassifier()))
models.append(('Gradient Boosting Classifier',GradientBoostingClassifier()))
models.append(('XGB Classifier', XGBClassifier()))
models.append(('Decision Tree Classifier', DecisionTreeClassifier()))
models.append(('LGBM Classifier', LGBMClassifier()))
models.append(('Support Vector Classifier', SVC()))
models.append(('KNN Classifier', KNeighborsClassifier()))

for name, model in models:
    model.fit(X_train, y_train)

    predictions = model.predict(X_test)
    accuracy = accuracy_score(predictions, y_test)

    print('Accuracy for {} : {:3.3f}\nClassification Report for {} : \n{}'.format(name, accuracy_score(predictions, y_test), name, classification_report(predictions, y_test, target_names=target_names)))

Since multiple models perform at same level, we can use a voting classifier

In [ ]:
from sklearn.ensemble import VotingClassifier

vote_models = []
vote_models.append(('Random Forest Classifier',RandomForestClassifier()))
vote_models.append(('Gradient Boosting Classifier',GradientBoostingClassifier()))
vote_models.append(('XGB Classifier', XGBClassifier()))
# vote_models.append(('LGBM Classifier', LGBMClassifier()))

In [ ]:
models = []
models.append(("Hard Vote",VotingClassifier(estimators = vote_models, voting ='hard')))
models.append(("Soft Vote",VotingClassifier(estimators = vote_models, voting ='soft')))

In [ ]:
for name,model in models:
    model.fit(X_train, y_train)
    
    predictions = model.predict(X_test)
    accuracy = accuracy_score(predictions, y_test)

    print('Accuracy for {} : {:3.3f}\nClassification Report for {} : \n{}'.format(name, accuracy_score(predictions, y_test), name, classification_report(predictions, y_test, target_names=target_names)))

In [ ]:
import pickle

In [ ]:
filename = 'xgboost_music_model.sav'
pickle.dump(models[1], open(filename, 'wb'))